In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from tensorflow.keras.models import load_model
import os

# Load your trained model
model = load_model("my_gru_model.h5")

N_CH = 55
WINDOW_SIZE = 5
STEP = 2

label_to_code = {0:1, 1:2, 2:5, 3:6}

def make_windows(epoch, window=WINDOW_SIZE, step=STEP):
    n_ch, n_t = epoch.shape
    data = epoch.T  # shape (time, channels)
    Xw = []
    for start in range(0, n_t - window + 1, step):
        Xw.append(data[start:start+window, :])
    return np.stack(Xw, axis=0).astype('float32')

def predict_event(epoch):
    ep = epoch.astype('float32').reshape(1, N_CH, -1)  # (1, channels, times)
    Xw = make_windows(ep[0])                            # (n_windows, window, channels)
    probs = model.predict(Xw, verbose=0)                # (n_windows, 4)
    preds = probs.argmax(axis=1)                         # predictions per window
    top = Counter(preds).most_common(1)[0][0]           # majority vote
    return label_to_code[top]

saved_trials_dir = "saved_trials"

for filename in os.listdir(saved_trials_dir):
    if filename.endswith(".csv"):
        filepath = os.path.join(saved_trials_dir, filename)
        df = pd.read_csv(filepath)
        if df.shape[1] != N_CH:
            raise ValueError(f"File {filename} has {df.shape[1]} channels, expected {N_CH}")
        trial_data = df.values.T.astype(np.float32)  
        pred_event = predict_event(trial_data)
        print(f"File: {filename}, Predicted event code: {pred_event}")


File: event_code_0_trial.csv, Predicted event code: 1
File: event_code_1_trial.csv, Predicted event code: 2
File: event_code_2_trial.csv, Predicted event code: 5
File: event_code_3_trial.csv, Predicted event code: 6
